##### 1. AutoTokenizer와 AutoModelForSeq2SeqLM
- AutoTokenizer: 입력 텍스트를 모델이 이해할 수 있는 토큰 ID로 변환하거나, 생성된 토큰 ID를 다시 텍스트로 변환하는 데 사용됩니다.
- AutoModelForSeq2SeqLM: T5 기반의 모델로, 입력 텍스트를 받아 요약된 결과를 출력합니다.


##### 2. tokenizer(text=input, ...)
- 입력 텍스트를 토큰화합니다.
- max_length=1024: 최대 1024개의 토큰으로 제한합니다. 텍스트가 길면 잘립니다.
- truncation=True: 최대 길이를 초과하면 자릅니다.
- return_tensors="pt": PyTorch 텐서로 반환합니다.
- 결과: 입력 텍스트가 토큰 ID로 변환되어 모델에 전달됩니다.

##### 3. model.generate(...)
- input_ids=input_ids: 모델이 이해할 수 있는 토큰 ID 입력.
- num_beams=5: 빔 서치(Beam Search) 기법 사용으로 더 나은 문장 생성.
- do_sample=True: 생성 결과를 확률적으로 샘플링.
- min_length=10, max_length=64: 생성할 요약 텍스트의 최소 및 최대 길이.

In [1]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')

input = """졸업식이 취소되어서 너무 아쉽다. 취소 이유는 중국 우한에서 처음 발병한 신종 코로나 바이러스 때문이다. 인생 마지막 졸업식인데 이렇게 취소가 되어버리다니 너무 허무하다. 사실은 졸업식에 별로 가고 싶지 않았다. 강의실에 우르르 모여 앉아 형식적인 학생회장의 말을 듣고, 그저 강의하는 사람 그 이상 그 이하도 아니었던 교수님들과 어색한 포옹을 하고 악수를 하며 학위기를 받는 그 상황이 너무 민망하고 어색할 것 같아서 가고 싶은 마음이 없었다. 그러다 마음을 바꾸게 된 건 우리 엄마의 부탁 때문이었다. 원래대로라면 2년 전에 이미 졸업을 했었어야 하지만 휴학, 유예 등으로 졸업이 2년이나 밀렸다. 2년을 기다려주시고, 이제 50대가 된 우리 엄마가 '더 늙기 전에 학사모 쓰고 사진 찍고 싶어!'라며 귀여운 부탁을 했다. 끝까지 거절하려다가 정말 얼마전에 엄마한테 졸업식 가자고 말씀드렸고, 엄마도 졸업식 날에 맞춰 연차까지 쓰셨지만 타이밍도 참. 속상하다. 팔정도에 있는 코끼리는 졸업식 때에만 올라탈 수 있다는 소문이 있다. 처음엔 갈 생각 없었던 졸업식이지만, 갈 마음을 먹고 나니 학사모 쓰고 코끼리 올라 탈 생각에 설레었는데 졸업식이 아예 취소가 되어버리니까 아쉬운 마음이 드는 건 왜일까? 8월에 있을 가을 학위수여식과 함께 진행한다고 하는데, 어느 누가 그 때를 기억하고 가겠어. 나는 괜찮은데, 엄마가 너무 아쉬워해서 마음이 쓰인다. 대학 입학부터 졸업까지 아무탈 없이, 걱정 없이 학교 생활 할 수 있게 도와주신 부모님께 졸업식 날에 맞춰서 꽃다발 선물이라도 해드려야겠다는 생각을 했다. 부모님 덕분에 좋은 학교 다니며, 더 멋진 사람으로 성장할 수 있게 도와주신 것에 대한 감사의 표현을 하는 것은 당연한 것 아닌가? '조금만 더 열심히 할 걸' 아쉬움이 남는 4년의 학교생활이었지만, 부모님 곁을 떠나 처음으로 타지에서 혼자 생활을 해보는 것부터 시작해서 처음 경험해볼 수 있었던 것들이 많아 행복했다. 물론 행복만 한 건 아니었지만, 대학 생활이 좋은 기억으로만 남지는 않겠지만, 그래도 소중했던 4년의 시간 덕분에 스스로 성장할 수 있었다.
"""
# 중간에 줄바꿈 없이 한 줄로 만드는 것이 좋다. 

# Task Prompt 추가
task_prompt = "감정 표현이 반영된 두 줄 요약: "
modified_input = task_prompt + input

input_ids = tokenizer(text=modified_input, max_length=1024, truncation=False, return_tensors="pt")['input_ids'] 

output = model.generate(input_ids=input_ids, num_beams=5, do_sample=True, min_length=10, max_length=64) #do_sample: 다음 토큰 선택을 확률 분포를 이용하여 샘플링한다.

tokenizer.batch_decode(output, skip_special_tokens=True)


c:\labs_python\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


['중국 우한에서 처음 발병한 신종 코로나 바이러스 때문에 인생 마지막 졸업식이 취소되었다.']

In [2]:
import pandas as pd

diary = pd.read_csv('감정_분류/가공데이터/merged_diary.csv', encoding='cp949')

In [3]:
diary

,author_age,author_sex,title,text_date,sentence
0,24,F,졸업,20200000,졸업식이 취소되어서 너무 아쉽다. 취소 이유는 중국 우한에서 처음 발병한 신종 코로...
1,61,M,58년 개띠,20200000,살 다 보니 어느 새 현역에서 은퇴하여 손주를 보며 즐거워하는 나이가 되었다. 지난...
2,27,F,일하기싫다,20200000,일이 하기 싫다. 오늘은 늦게 일어나서 지각을 할 뻔했지만 지각을 하지 않았다. 다...
3,31,F,만약에 내가 로또에 당첨 된다면,20200000,만약에 내가 로또에 당첨된다면 난 그 돈으로 무엇을 할지 가끔 고민해본다. 로또에...
4,39,F,40인생,20200000,어른들의 말이 문득 떠오른다... 크면 다 알게될꺼라는 말들 이제야 문득문득 어릴때...
...,...,...,...,...,...
4200,7,F,할머니 집에 간 날,20200000,<할머니 집에 간 날> 나는 오늘 할머니네 집에 놀러 갔다.내가 수족구에 걸려서 나...
4201,8,M,미사수변공원,20190810,8월 10일 금 요일 날씨: 불 타 오르도록더운날 나는 오늘 친구와 아빠동생나 하고...
4202,8,M,도서관가학원,20190807,8월 7일 수요일 날씨:변덕스러운날씨 나는오 늘 도서관가학원 을 갔 다 온디 엄마 ...
4203,8,M,할아버지와할머니 생신,20190803,8월 3일 날씨:쨍쨍하 면 시비가 많이온날 오늘은 할아버지와할머니가 생신을 같 이 ...


In [9]:
diary2 = diary[700:1000].copy()

In [5]:
max([len(i) for i in diary2['sentence']])

7715

In [10]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained('eenzeenee/t5-base-korean-summarization')
tokenizer = AutoTokenizer.from_pretrained('eenzeenee/t5-base-korean-summarization')

def generate_summary(text, task_prompt="감정 표현이 반영된 두 줄 요약:"):
    # 입력 텍스트 전처리
    text = text.replace('\n', ' ').strip()
    
    # 프롬프트 형식 지정
    modified_input = f"{task_prompt} {text}"
    
    # 토크나이징
    inputs = tokenizer(
        modified_input,
        max_length=2048,  # 입력 텍스트의 최대 길이 제한
        truncation=True,   # max_length를 초과하는 텍스트는 자동으로 자름
        padding='longest',   # 배치 내에서 가장 긴 시퀀스에 맞춰 패딩
        return_tensors="pt"   # PyTorch 텐서 형태로 반환
    )
    
    # 요약 생성
    outputs = model.generate(
        input_ids=inputs['input_ids'],  # 토큰화된 입력 텍스트
        attention_mask=inputs['attention_mask'],  # 패딩된 부분을 무시하기 위한 마스크
        # 생성 품질 관련
        num_beams=5,  # 빔 서치 크기 (더 높은 값 = 더 다양한 후보 고려)
        length_penalty=2.0,   # 길이에 대한 페널티 (>1: 긴 문장 선호, <1: 짧은 문장 선호)
        max_length=200,     # 생성될 텍스트의 최대 길이
        min_length=10,      # 생성될 텍스트의 최소 길이
        # 반복 방지
        no_repeat_ngram_size=2,
        do_sample=True,      # 확률적 샘플링 사용 (True: 더 다양한 출력)
        top_k=50,        # 각 단계에서 고려할 최상위 k개의 토큰
        top_p=0.95,     # 누적 확률이 p가 될 때까지의 토큰만 고려
        early_stopping=True   # 모든 빔이 종료 토큰에 도달하면 생성 중단
    )
    
    # 디코딩
    summary = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return summary

# 사용 예시
input_text = 'sentence'

# 다양한 프롬프트로 시도
prompts = [
    # "감정이 드러나게 요약:",
    "감정을 포함하여 핵심 내용 요약:"
    #"주요 감정과 사건을 요약:",
    # "감정과 상황을 중심으로 요약:"
]

diary2['sentence_summary'] = diary2['sentence'].apply(lambda x: generate_summary(x, prompts[0]))

# for prompt in prompts:
    # summary = generate_summary(input_text, prompt)
    # print(f"\n{prompt}\n{summary}")

diary2.to_csv('일기_요약본700-999.csv', index=False)